In [2]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

In [3]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path
from sklearn.model_selection import train_test_split
#path append
sys.path.append(os.path.join(Path.cwd(), './modules'))
import Evaluate, Helper, Present, Score, Synthetic, Modeling


In [4]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = './config/config.yaml'
prediction_config_files = ['./config/predictions/pred1.yaml']
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

In [5]:

base_dir = Path.cwd() #For running locally
#base_dir = cwd_dir.parent
data_dir= base_dir / 'data' /'simulated_health_data' / 'data' 


Let's provide some examples of working with partitioned parquet files. 

In [6]:
health=pd.read_csv(data_dir / 'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.csv')
health

,person_id,yyyymm,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
0,1000,201701,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1,1000,201702,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
2,1000,201703,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
3,1000,201704,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
4,1000,201705,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215995,9999,201808,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215996,9999,201809,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215997,9999,201810,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215998,9999,201811,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99


In [7]:
referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
referrals

,PERSON_ID,MYR
0,30,2017-01-29
1,53,2016-05-17
2,120,2016-02-27
3,154,2016-08-31
4,158,2016-08-29
5,204,2017-09-29
6,233,2017-08-10
7,279,2016-11-27
8,294,2016-08-20
9,325,2017-04-19


In [7]:
#Need to match the Health data with the referral.  
#NAFIS TODO
#Prior referrals should be incorporated as features. 
#Should be able to use similar logic to the evaluation merge. 1 if it becomes referral in any months in the next window. 
#Instead going to continue 
#I think we should try to do 1 month for training and another month for testing. 

In [8]:
#select out one month and set index equal to the personid
health=health.loc[health['yyyymm']==201712,:]
health.index=health['person_id']
health

,person_id,yyyymm,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
person_id,,,,,,,,,,,,,,,,,,,,,
1000,1000,201712,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1001,1001,201712,1,1,1,1,1,1,1,1,...,-0.933237,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1
1002,1002,201712,1,1,1,1,1,1,1,1,...,-0.134841,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2
1003,1003,201712,1,1,1,1,1,1,1,1,...,-0.943406,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3
1004,1004,201712,1,1,1,1,1,1,1,1,...,1.025984,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,201712,1,1,1,0,0,0,0,0,...,-0.814544,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95
9996,9996,201712,1,1,1,0,0,0,0,0,...,-0.308765,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96
9997,9997,201712,1,1,1,0,0,0,0,0,...,-0.121841,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97


In [9]:
#TRAINTEST SPLIT
y=health['target_bin']
y_class=health['target_class']
X=health[['cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]

In [10]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.5,
                                                    test_size=0.5,
                                                    random_state=123,
                                                    stratify=y)

In [22]:
test_X

,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,cad9,dv0,dv1,dv2,dv3,dv4,dv5,dv6,dv7,dv8,dv9
person_id,,,,,,,,,,,,,,,,,,,,
1635,0,0,1,0,0,0,0,1,1,0,-1.883809,-1.314142,2.601240,-0.612552,-1.714963,1.350046,1.798990,-1.312854,0.033590,-0.220208
1682,0,0,1,0,0,0,0,1,1,0,-0.212220,1.330096,0.683211,-0.008983,0.429693,0.662408,1.223457,0.842224,2.133137,1.159285
4285,1,1,1,1,0,1,1,0,0,1,1.000157,2.014163,-0.051793,-0.024450,-0.336471,0.150686,-1.176741,0.033326,0.420188,-0.072864
2661,1,0,0,1,0,1,1,1,0,1,-1.610592,-0.279079,-1.626532,-0.596513,1.411114,-0.306438,-0.363666,-0.367746,-0.641860,0.362993
4553,1,1,0,0,1,0,0,0,1,0,-0.165657,-0.882452,0.264956,-1.111217,0.838148,0.115902,-0.436291,-0.546872,0.827631,-0.230527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8792,0,0,1,1,0,0,1,1,1,0,-1.329598,-0.274925,-0.052349,0.966609,-1.706763,-0.580659,1.156626,-0.470667,-0.147510,-2.257347
5843,1,1,1,0,1,1,0,1,1,0,-0.790135,1.907207,-0.701428,0.776298,0.447891,0.812578,0.379137,2.067828,0.355468,-0.246497
2762,0,1,0,0,1,1,0,0,1,1,-0.311174,0.118887,0.012922,-0.706988,0.646012,0.503006,0.972973,0.608365,-1.371911,-1.390943


In [24]:
train_predict, test_predict, trained_models= Modeling.perform_classification(train_X, test_X, train_y, test_y)

           target
person_id        
9705            0
7243            0
6852            0
4167            1
3230            1
...           ...
5396            0
1688            0
3354            0
3982            0
3341            1

[4500 rows x 1 columns]            target
person_id        
1635            0
1682            1
4285            1
2661            0
4553            0
...           ...
8792            0
5843            1
2762            0
8564            0
7499            0

[4500 rows x 1 columns]
Fitting model:  k_nearest_neighbors_10
Fitting model:  svm_linear
Fitting model:  svm_nonlinear
Fitting model:  decision_tree
Fitting model:  naive_bayes
Fitting model:  random_forest
Fitting model:  adaboost
Fitting model:  quadratic_discriminant_analysis


In [21]:
test_predict

,target,k_nearest_neighbors_10,svm_linear,svm_nonlinear,decision_tree,naive_bayes,random_forest,adaboost,quadratic_discriminant_analysis
person_id,,,,,,,,,
1635,0,0.0,0.000655,1.594521e-04,0.0,0.005911,0.01,0.442895,5.334624e-05
1682,1,0.3,0.733001,1.000000e+00,1.0,0.428256,0.86,0.517489,8.690149e-01
4285,1,0.6,0.635088,1.000000e+00,1.0,0.440287,1.00,0.496106,7.959353e-01
2661,0,0.0,0.001145,1.000000e-07,0.0,0.026120,0.00,0.459586,2.532655e-05
4553,0,0.0,0.007951,5.365940e-05,0.0,0.062092,0.00,0.470665,4.330735e-03
...,...,...,...,...,...,...,...,...,...
8792,0,0.0,0.001450,1.000000e-07,0.0,0.016670,0.00,0.447351,7.303137e-05
5843,1,0.9,0.544469,9.999998e-01,1.0,0.368506,0.97,0.508206,6.541873e-01
2762,0,0.0,0.049139,3.519372e-01,0.0,0.116714,0.00,0.481844,3.609189e-02
